## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
import pandas as pd

In [27]:
film_data = pd.read_csv('ml-latest-small/ratings.csv')
film_data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [28]:
# у каждого фильма есть много оценок, поэтому, сначала посчитаем среднюю оценку для каждого
film_rating = film_data.groupby('movieId').mean()[['rating']].reset_index()
film_rating.head()

,movieId,rating
0,1,3.872470
1,2,3.401869
2,3,3.161017
3,4,2.384615
4,5,3.267857


In [29]:
def group_by_class(row):
    if row.rating <= 2:
        return 'Low'
    elif 2 <= row.rating <= 4:
        return 'Middle'
    elif row.rating == 4.5 or row.rating == 5:
        return 'High'

film_rating['class'] = film_rating.apply(group_by_class, axis=1)
film_rating.sort_values('rating').head()

,movieId,rating,class
7592,80350,0.5,Low
4340,5850,0.5,Low
6524,48591,0.5,Low
6522,48522,0.5,Low
4345,5864,0.5,Low


## Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [34]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [35]:
key_data = pd.read_csv('keywords.csv')
key_data.head(10)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
5,одноклассники моя страница,19976138
6,майл,15144673
7,авито,14786593
8,переводчик,14453469
9,яндекс,13344734


In [81]:
for region in geo_data.items():
    print(region[1])

['москва', 'тула', 'ярославль']
['петербург', 'псков', 'мурманск']
['владивосток', 'сахалин', 'хабаровск']


In [82]:
def find_geo(row):
    for region in geo_data.items():
        if any(city in row.keyword for city in region[1]):
            return region[0]

In [83]:
key_data['region'] = key_data.apply(find_geo, axis=1)
key_data.head()

# проверка провильности
# key_data[key_data['keyword'].str.contains('тула')].head()

,keyword,shows,region
0,вк,64292779,None
1,одноклассники,63810309,None
2,порно,41747114,None
3,ютуб,39995567,None
4,вконтакте,21014195,None


## Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга